# Analiza datelor

 - ## class - Coloana tinta

    - sit and bend forward_cm - Are ceea mai mare corelatie cu coloana tinta
      - Dupa parerea mea are asa o corelatie mare pentru ca ideia la dataset e sa determine nivelul de a putea lucra cu anii. Asa dar ceea cat de departe poti sa te apleci arata cat de elastic esti si in acelas moment cat de bine vei indepleni lucrul. Cat mai elastic esti cu atat mai mult lucru poti face intr-un fel

In [169]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dataset = pd.read_csv('bodyPerformance.csv')

dataset['gender'] = dataset['gender'].map({'F': 0, 'M': 1})
dataset['class'] = dataset['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3})

corr = dataset.corr()
max_corr = corr['class'].abs().nlargest(2)

#dataset['class'].value_counts()
#dataset['gender'].value_counts()
#max_corr 
dataset

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,1,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,2
13389,21.0,1,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,3
13390,39.0,1,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,0
13391,64.0,0,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,3


## Dataset cu corelatia intre 0.5 si 0.8 cu coloana tinta

In [170]:
interval = corr[(corr['class'].abs() > 0.5) & (corr['class'].abs() < 0.8)]

new_dataset = dataset[interval.index]
new_dataset['class'] = dataset['class']

#new_dataset

C:\Users\Home\AppData\Local\Temp\ipykernel_432\1575015547.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['class'] = dataset['class']


## Amestecarea dataset-ului

In [171]:
dataset_values, columns = dataset.values, list(dataset.columns)
np.random.shuffle(dataset_values)
dataset = pd.DataFrame(dataset_values, columns=columns)

new_dataset_values, columns = new_dataset.values, list(new_dataset.columns)
np.random.shuffle(new_dataset_values)
new_dataset = pd.DataFrame(new_dataset_values, columns=columns)

#dataset

## Impartirea in tren si test cu sklearn

In [181]:
X = dataset.drop(columns=['class'])
y = dataset['class']

X2 = new_dataset.drop(columns=['class'])
y2 = new_dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=42)

#X_train

# Antrenarea modelului Logistic Regression de la sklearn

In [188]:
model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

model2 = LogisticRegression(random_state=0)
model2.fit(X_train2, y_train2)

C:\Users\Home\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

## Rezultatul modelului

In [189]:
y_pred = pd.DataFrame(model2.predict(X_test2), columns=['class'])
y_pred2 = pd.DataFrame(model2.predict(X_test2), columns=['class'])

score = accuracy_score(y_true=y_test, y_pred=y_pred)
score2 = accuracy_score(y_true=y_test2, y_pred=y_pred2)

print(y_pred['class'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'}).value_counts())
y_pred2['class'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'}).value_counts()
#score

class
A    1269
D    1123
B     859
C     767
Name: count, dtype: int64


class
A    1269
D    1123
B     859
C     767
Name: count, dtype: int64

# Implementarea de la 0 a Logistic Regression din carte


In [197]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate: float = 0.05, max_iter: int = 5000) -> None:
        '''
        The constructor of the Logistic Regression model.
        :param learning_rate: float, default=0.05
            The learning rate of the model.
        :param max_iter: int, default=100000
            The number of iterations to go through.
        '''
        # Setting up the hyperparameters.
        self.__learning_rate = learning_rate
        self.__max_iter = max_iter

    def sigmoid(self, y: np.array) -> np.array:
        '''
        The sigmoid function.
        :param y: np.array
            The predictions of the linear function.
        '''
        return 1 / (1 + np.exp(-y))

    def fit(self, X: np.array, y: np.array):
        '''
        The fit function of the model.
        :param X: 2-d np.array
            The matrix with the features.
        :param y: 1-d np.array
            The target vector.
        '''
        # Creating the weights vector.
        self.coef_ = np.zeros(len(X[0]) + 1)
        # Adding the intercept column.
        X = np.hstack((X, np.ones((len(X), 1))))
        # The weights updating process.
        for i in range(self.__max_iter):
            # Prediction.
            pred = self.sigmoid(np.dot(X, self.coef_))
            # Computing the gradient.
            gradient = np.dot(X.T, (pred - y)) / y.size
            # Updating the weights.
            self.coef_ -= gradient * self.__learning_rate
        return self

    def predict_proba(self, X: np.array) -> np.array:
        '''
        This function returns the class probabilities.
        :param X: 2-d np.array
            The features matrix.
        :return: 2-d, np.array
            The array with the probabilities for every class for every sample.
        '''
        # Adding the intercept column.
        X = np.hstack((X, np.ones((len(X), 1))))
        # Computing the probabilities.
        prob = self.sigmoid(np.dot(X, self.coef_))
        # Returning the probabilities.
        return np.hstack(((1 - prob).reshape(-1, 1), prob.reshape(-1, 1)))

    def predict(self, X: np.array) -> np.array:
        '''
        This function returns the predictions of the model.
        :param X: 2-d np.array
            The features matrix.
        :return: 2-d, np.array
            The array with the probabilities for every class for every sample.
        '''
        # Adding the intercept column.
        X = np.hstack((X, np.ones((len(X), 1))))
        return (self.sigmoid(np.dot(X, self.coef_)) > 0.5) * 1
    
lr = LogisticRegression()
lr.fit(X=X_train.values, y=y_train.values)

lr2 = LogisticRegression()
lr2.fit(X=X_train2.values, y=y_train2.values)


C:\Users\Home\AppData\Local\Temp\ipykernel_432\3106041102.py:22: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-y))


## Rezultatul modelului

In [198]:
y_pred2_1 = pd.DataFrame(lr.predict(X_test), columns=['class'])
y_pred2_2 = pd.DataFrame(lr2.predict(X_test2), columns=['class'])

score2_1 = accuracy_score(y_true=y_test, y_pred=y_pred2_1)
score2_2 = accuracy_score(y_true=y_test2, y_pred=y_pred2_2)

#y_pred2_1['class'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'}).value_counts() #WHAT?
y_pred2_2['class'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'}).value_counts()
#score

C:\Users\Home\AppData\Local\Temp\ipykernel_432\3106041102.py:22: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-y))


class
B    3845
A     173
Name: count, dtype: int64